# DOES NOT WORK UNLESS NUMPY 1.26 IS INSTALLED

Create a new environment and specify: 'pip install numpy=1.26' in order to use this notebook.

# Drainage Networks from DEM Data using pysheds

In [ ]:
import os
import numpy as np
import geopandas as gpd
# Water shed calculations
from pysheds.grid import Grid
# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as colors
# Shapely
from shapely import geometry, ops
# Geojson
import geojson
# Folium
import folium

In [ ]:
# First we need to open our existing shapefiles
with open("../data/GIS/Barkley_Rivers.geojson", 'r') as river_file:
    be_rivers = geojson.load(river_file)
with open("../data/GIS/Barkley_SamplingPoints.geojson", 'r') as sampling_file:
    be_sampling = geojson.load(sampling_file)

map_be = folium.Map(location=[-31.0, 27.5])
folium.GeoJson(be_rivers).add_to(map_be)
folium.GeoJson(be_sampling).add_to(map_be)
folium.LayerControl().add_to(map_be)

map_be

In [ ]:
# Define input path
raster_input_file = "../data/GIS/output_AW3D30.tif"
# Create a grid from input file and load the file
dem_input_grid = Grid.from_raster(data=raster_input_file, data_name='dem', nodata= np.array([np.nan], dtype=np.float32)[0])
dem_input_file = dem_input_grid.read_raster(raster_input_file)

# Define a function to plot the digital elevation model
def plot_figure(data, grid, label):
    plt.figure()
    plt.imshow(data, extent=grid.extent, zorder=1)
    plt.colorbar(label='Elevation (m)')
    plt.grid(zorder=0)

# Plot the loaded raster
plot_figure(dem_input_file, dem_input_grid, 'Elevation (m)')

In [ ]:
# Detect depressions
depressions = dem_input_grid.detect_depressions(dem_input_file)

In [ ]:
# Fill pits
pit_filled_dem = dem_input_grid.fill_pits(dem_input_file)
# Fill depressions
flooded_dem = dem_input_grid.fill_depressions(pit_filled_dem)
# Resolve flats in the model
inflated_dem = dem_input_grid.resolve_flats(flooded_dem)
# Create a flow direction grid
#         N    NE    E    SE    S    SW    W    NW
dirmap = (64,  128,  1,   2,    4,   8,    16,  32)
# Calculate flow directions
flow_dir = dem_input_grid.flowdir(inflated_dem, dirmap=dirmap)
# Plot the flow directions
plot_figure(flow_dir, dem_input_grid, 'Flow Direction')

In [ ]:
# Calculate accumulation
accumulation = dem_input_grid.accumulation(flow_dir, dirmap=dirmap)
# Plot accumulation
fig_acc, ax_acc = plt.subplots(figsize=(8,6))
fig_acc.patch.set_alpha(0)
plt.grid('on', zorder=0)
im = ax_acc.imshow(accumulation, extent=dem_input_grid.extent, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, accumulation.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax_acc, label='Upstream Cells')
plt.title('Flow Accumulation', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

In [ ]:
# Specify discharge point
x, y = 26.7655, -30.7081

# Snap the discharge point to the high accumulation point on the grid
x_snap, y_snap = dem_input_grid.snap_to_mask(accumulation > 100000, (x, y)) #100k

# Delineate the catchment
catchment = dem_input_grid.catchment(fdir=flow_dir, x=x_snap, y=y_snap, dirmap=dirmap, xytype='label')

# Plot the catchment
dem_input_grid.clip_to(catchment)
clipped_catch = dem_input_grid.view(catchment)
plot_figure(clipped_catch, dem_input_grid, "None")

In [ ]:
# Calculate river branches
branches = dem_input_grid.extract_river_network(flow_dir, accumulation > 1000, dirmap=dirmap) #10k
# Create a figure for the branches
fig_branches, ax_branches = plt.subplots(figsize=(8.5,6.5))
plt.xlim(dem_input_grid.bbox[0], dem_input_grid.bbox[2])
plt.ylim(dem_input_grid.bbox[1], dem_input_grid.bbox[3])
ax_branches.set_aspect('equal')
# Plot the branches
for branch in branches['features']:
    line = np.asarray(branch['geometry']['coordinates'])
    plt.plot(line[:, 0], line[:, 1])
# Set the figure title
_ = plt.title('D8 channels', size=14)

In [ ]:
out_file_geojson = '../output/rivers_extracted.geojson'

with open(out_file_geojson, 'w') as outfile:
     geojson.dump(branches, outfile)

dfBranches = gpd.read_file(out_file_geojson)

dfBranches.explore(tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google Earth')

In [ ]:
map_be_mod = folium.Map(location=[-31.0, 27.5])
folium.GeoJson(be_rivers).add_to(map_be_mod)
folium.GeoJson(be_sampling).add_to(map_be_mod)
folium.GeoJson(branches).add_to(map_be_mod)
folium.LayerControl().add_to(map_be_mod)

map_be_mod

In [ ]:
map_be_mod.save("../output/rivers.html")